In [1]:
import os
import json
import pandas as pd
from annotation_config import *

In [ ]:
result_path = 'batch_5_src_no_sports/results'
result_files = []
skip_sample_ids = {}

for batch_id in range(1,16+1):
    file_name = os.path.join(result_path, f"batch_{batch_id}_annotation.json")
    result_files.append(file_name)
    if batch_id in skip_samples:
        skip_sample_ids[file_name] = skip_samples[batch_id]
final_data = {'source':[], 'summary':[], 'LLM': [], 'worst-label':[], 'best-label':[]}
for file_path in result_files:        
    data = json.load(open(file_path))
    sample_count = 0
    annotators = set()
    for sample in data:
        sample_id = sample['sample_id']
        if file_path in skip_sample_ids and sample_id in skip_sample_ids[file_path]:
            continue
        # model_name = sample['meta_model']
        annotations = sample['annotations']
        for annotation in annotations:
            annotators.add(annotation['annotator'])

    num_annotators = len(annotators)
    
    for sample in data: 
        final_data['source'].append(sample['source'])
        final_data['summary'].append(sample['summary'])
        final_data['LLM'].append(sample['meta_model'])
        sample_annotations = []
        occurred_annotators = set()
        annotations = sample['annotations']
        for annotation in annotations:
            # print(annotation)
            sample_annotations.extend(annotation['label'])
            occurred_annotators.add(annotation['annotator'])

        if len(occurred_annotators) < num_annotators:
            sample_annotations.extend(['Consistent']* (num_annotators - len(occurred_annotators)))

        sample_annotations = set(sample_annotations)
        # print(sample_annotations)
        # human annotation
        if "Unwanted" in sample_annotations:
            worst_sample_pred = "Unwanted"
        elif 'Questionable' in sample_annotations:
            worst_sample_pred = "Questionable"
        elif "Benign" in sample_annotations:
            worst_sample_pred = "Benign"
        else:
            worst_sample_pred = "Consistent"

        if "Consistent" in sample_annotations:
            best_sample_pred = "Consistent"
        elif "Benign" in sample_annotations:
            best_sample_pred = "Benign"
        elif 'Questionable' in sample_annotations:
            best_sample_pred = "Questionable"
        else:
            best_sample_pred = "Unwanted"
        # print('worst_sample_pred', worst_sample_pred)
        # print('best_sample_pred', best_sample_pred)
        final_data['worst-label'].append(worst_sample_pred)
        final_data['best-label'].append(best_sample_pred)

final_data_df = pd.DataFrame.from_dict(final_data, orient='columns') 
print(final_data_df.shape)
final_data_df.to_csv('FaithBench.csv', mode='w', index=False, header=True)
        
            

(800, 5)
